In [4]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [1]:
original = pd.read_csv("../datas/life_expectancy_data_4.29.csv")
original.columns = [cols.upper() for cols in original.columns.tolist()]
original["STATUS"] = [row.replace("Developing","0") for row in original["STATUS"].tolist()]
original["STATUS"] = [row.replace("Developed","1") for row in original["STATUS"].tolist()]
original["STATUS"] = [int(row) for row in original["STATUS"].tolist()]
original.head()

,COUNTRYCODE,ISO3166,COUNTRY,YEAR,STATUS,LIFE_EXPECTANCY,ADULT_MORTALITY,INFANT_DEATHS,ALCOHOL,PERCENTAGE_EXPENDITURE,...,POLIO,TOTAL_EXPENDITURE,DIPHTHERIA,HIV/AIDS,GDP,POPULATION,THINNESS_1_19_YEARS,THINNESS_5_9_YEARS,INCOME_COMPOSITION_OF_RESOURCES,SCHOOLING
0,AFG,4,Afghanistan,2015,0,65.0,263.0,62,0.01,71.279624,...,6.0,8.16,65.0,0.1,1.990711e+10,34413603,17.2,17.3,0.479,10.1
1,AFG,4,Afghanistan,2014,0,59.9,271.0,64,0.01,73.523582,...,58.0,8.18,62.0,0.1,2.048489e+10,33370794,17.5,17.5,0.476,10.0
2,AFG,4,Afghanistan,2013,0,59.9,268.0,66,0.01,73.219243,...,62.0,8.13,64.0,0.1,2.056107e+10,32269589,17.7,17.7,0.470,9.9
3,AFG,4,Afghanistan,2012,0,59.5,272.0,69,0.01,78.184215,...,67.0,8.52,67.0,0.1,2.000160e+10,31161376,17.9,18.0,0.463,9.8
4,AFG,4,Afghanistan,2011,0,59.2,275.0,71,0.01,7.097109,...,68.0,7.87,68.0,0.1,1.780429e+10,30117413,18.2,18.2,0.454,9.5


In [2]:
cat = {'economy': ['STATUS', 'PERCENTAGE_EXPENDITURE', 'TOTAL_EXPENDITURE', 'GDP', 'POPULATION'],
        'death_rate': ['INFANT_DEATHS', "ADULT_MORTALITY",'UNDER_FIVE_DEATHS','THINNESS_1_19_YEARS','THINNESS_5_9_YEARS', 'HIV/AIDS'],
        'vaccine': ['HEPATITIS_B','MEASLES','POLIO','DIPHTHERIA'],
        'others':['SCHOOLING','INCOME_COMPOSITION_OF_RESOURCES', 'BMI', 'ALCOHOL']}

In [7]:
missing = original.isnull().sum() 
missing = missing[missing > 0] 
missing.sort_values(inplace=True) 

fig = px.bar(missing)
fig.update_layout(
    title_text='Life Expectancy Data의 컬럼별 결측 비중',
    height=500,
    width=600,
    template='plotly_white'
)

fig.show()

### Population 결측치 보간
- Eritrea 2012~2015년 인구 0명 결측치 2011년 인구로 대체

In [59]:
#ERI' 인구 2012 ~ 2015년 0명 > 2011년 321만명으로 대체
replace_to =  original[(original["COUNTRYCODE"]=='ERI') & (original["YEAR"]==2011)]["POPULATION"]
original['POPULATION'].replace(0, int(replace_to),inplace=True)

In [69]:
# original['POPULATION'].replace(0, int(replace_to),inplace=True)
# original[(original["COUNTRYCODE"]=='ERI')]

,COUNTRYCODE,ISO3166,COUNTRY,YEAR,STATUS,LIFE_EXPECTANCY,ADULT_MORTALITY,INFANT_DEATHS,ALCOHOL,PERCENTAGE_EXPENDITURE,HEPATITIS_B,MEASLES,BMI,UNDER_FIVE_DEATHS,POLIO,TOTAL_EXPENDITURE,DIPHTHERIA,HIV/AIDS,GDP,POPULATION,THINNESS_1_19_YEARS,THINNESS_5_9_YEARS,INCOME_COMPOSITION_OF_RESOURCES,SCHOOLING
849,ERI,232,Eritrea,2015,0,64.7,255.0,5,NaN,0.000000,95.0,198,18.6,7,95.0,NaN,95.0,0.4,NaN,3213972,8.4,8.2,0.418,5.0
850,ERI,232,Eritrea,2014,0,64.4,261.0,5,0.01,0.000000,94.0,127,18.0,8,94.0,3.34,94.0,0.4,NaN,3213972,8.5,8.3,0.416,5.0
851,ERI,232,Eritrea,2013,0,64.0,266.0,6,0.01,0.000000,94.0,45,17.5,8,94.0,3.10,94.0,0.5,NaN,3213972,8.6,8.4,0.414,5.0
852,ERI,232,Eritrea,2012,0,63.6,274.0,6,0.01,0.000000,94.0,194,17.0,8,94.0,2.98,94.0,0.6,NaN,3213972,8.7,8.6,0.410,5.0
853,ERI,232,Eritrea,2011,0,62.9,286.0,6,0.62,20.979919,96.0,48,16.5,8,96.0,3.60,96.0,0.7,2.065002e+09,3213972,8.8,8.7,0.405,5.0
854,ERI,232,Eritrea,2010,0,62.1,298.0,6,0.61,17.357398,9.0,51,16.0,9,9.0,3.24,9.0,0.9,1.589515e+09,3170435,8.9,8.8,0.404,5.1
855,ERI,232,Eritrea,2009,0,61.4,311.0,6,0.63,1.575160,92.0,82,15.5,9,92.0,3.30,92.0,1.0,1.856696e+09,3119920,9.0,8.9,0.402,5.2
856,ERI,232,Eritrea,2008,0,67.0,322.0,6,0.49,11.765723,94.0,0,15.1,9,94.0,3.69,94.0,1.1,1.380189e+09,3062779,9.1,9.1,0.406,5.2
857,ERI,232,Eritrea,2007,0,62.0,329.0,6,1.23,11.423860,91.0,55,14.7,9,91.0,3.29,91.0,1.3,1.317974e+09,2996536,9.2,9.2,0.405,5.3
858,ERI,232,Eritrea,2006,0,59.7,336.0,7,0.97,10.602698,94.0,128,14.3,9,94.0,3.30,94.0,1.4,1.211162e+09,2918205,9.3,9.3,0.405,5.3


### GDP 결측치 보간
- DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간
- interpolate DataFrame missing values linearly
- 출처: [링크 참조](https://rfriend.tistory.com/264?category=675917)

In [70]:
GDP_null_country = original[original['GDP'].isnull()]["COUNTRYCODE"].unique()
GDP_null_country.tolist()

['AFG', 'PRK', 'ERI', 'IRQ', 'STP', 'SOM', 'SSD', 'SYR', 'VEN']

In [71]:
# GDP에 결측값이 있는 국가들 GDP 및 POPULATION 출력
original[original["COUNTRYCODE"].isin(GDP_null_country.tolist())][["COUNTRYCODE", 'YEAR', 'GDP','POPULATION']]

,COUNTRYCODE,YEAR,GDP,POPULATION
0,AFG,2015,1.990711e+10,34413603
1,AFG,2014,2.048489e+10,33370794
2,AFG,2013,2.056107e+10,32269589
3,AFG,2012,2.000160e+10,31161376
4,AFG,2011,1.780429e+10,30117413
5,AFG,2010,1.585657e+10,29185507
6,AFG,2009,1.243909e+10,28394813
7,AFG,2008,1.010923e+10,27722276
8,AFG,2007,9.747880e+09,27100536
9,AFG,2006,6.971286e+09,26433049


In [88]:
for c in GDP_null_country.tolist():
    df = original[original["COUNTRYCODE"]==c][["COUNTRYCODE", 'YEAR', 'GDP','POPULATION']]
    df.interpolate(method='linear', inplace=True)

In [89]:
df

,COUNTRYCODE,YEAR,GDP,POPULATION
2856,VEN,2015,NaN,30081829
2857,VEN,2014,4.823590e+11,30045134
2858,VEN,2013,3.710050e+11,29783571
2859,VEN,2012,3.812860e+11,29362449
2860,VEN,2011,3.164820e+11,28888369
2861,VEN,2010,3.931920e+11,28439940
2862,VEN,2009,3.297880e+11,28031009
2863,VEN,2008,3.159530e+11,27635832
2864,VEN,2007,2.303640e+11,27247610
2865,VEN,2006,1.834780e+11,26850194


In [79]:
original[original["COUNTRYCODE"].isin(GDP_null_country.tolist())][["COUNTRYCODE", 'YEAR', 'GDP','POPULATION']]

,COUNTRYCODE,YEAR,GDP,POPULATION
0,AFG,2015,1.990711e+10,34413603
1,AFG,2014,2.048489e+10,33370794
2,AFG,2013,2.056107e+10,32269589
3,AFG,2012,2.000160e+10,31161376
4,AFG,2011,1.780429e+10,30117413
5,AFG,2010,1.585657e+10,29185507
6,AFG,2009,1.243909e+10,28394813
7,AFG,2008,1.010923e+10,27722276
8,AFG,2007,9.747880e+09,27100536
9,AFG,2006,6.971286e+09,26433049
